In [1]:
import sys
sys.path.append('../src')
from utils import EventStream
from model_buffers import OcdfgBuffer, OcpnBuffer, TotemBuffer
from monitor import CacheMonitor, RuntimeMonitor
from monitor_evaluation import create_all_plots_cache_monitor, create_all_plots_runtime_monitor
from cache_policy_buffers import CachePolicy
from priority_policy_buffers import PrioPolicyOrder, PPBStridePerObjectType, PPBEventsPerObjectType
from model_builder_totem import visualize_totem_overlap, TotemModel

import pandas as pd
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
from pathlib import Path

In [2]:
# Load event stream
event_stream = EventStream('../data/ContainerLogistics.json')

# Define colors per object type for overlapped TOTeM visualization for replacement strategy with high vs. low accuracy
ots = sorted(event_stream.object_types)
ot_rgb_colors = cm.jet(np.linspace(0, 1, len(ots)))
ot_to_rgb_color = dict(zip(ots, ot_rgb_colors))
ot_to_hex_color = {ot: mpl.colors.rgb2hex(ot_rgb) for ot, ot_rgb in ot_to_rgb_color.items()}

### Evaluation of `RuntimeMonitor` on `ContainerLogistics.json`

In [ ]:
ocdfg_buf = OcdfgBuffer(
    node_buf_size=125, 
    arc_buf_size=125,
    cp=CachePolicy.LFU_DA,
    pp_buf=PPBStridePerObjectType(
        prio_order=PrioPolicyOrder.MAX, 
        freeze_or_max_idle=pd.Timedelta(days=1)
    ),
    coupled_removal=True,
    runtime_monitor=RuntimeMonitor()
)

ocpn_buf = OcpnBuffer(
    node_buf_size=125, 
    arc_buf_size=125,
    ea_buf_size=125,
    cp=CachePolicy.LFU_DA,
    pp_buf=PPBStridePerObjectType(
        prio_order=PrioPolicyOrder.MAX, 
        freeze_or_max_idle=pd.Timedelta(days=1)
    ),
    coupled_removal=True,
    runtime_monitor=RuntimeMonitor()
)

totem_buf = TotemBuffer(
    tr_buf_size=125, 
    ec_buf_size=125,
    lc_buf_size=125,
    cp=CachePolicy.LFU_DA,
    pp_buf=PPBStridePerObjectType(
        prio_order=PrioPolicyOrder.MAX, 
        freeze_or_max_idle=pd.Timedelta(days=1)
    ),
    coupled_removal=True,
    runtime_monitor=RuntimeMonitor()
)

for model_buf, model in [(totem_buf, 'totem'), (ocdfg_buf, 'ocdfg'), (ocpn_buf, 'ocpn')]:
    model_buf.process_stream(event_stream.stream)
    model_buf.create_monitor_dataframes()
    create_all_plots_runtime_monitor(model_buf.rt_mon, Path(f'../figures/ContainerLogistics/{model}_monitoring/{model_buf.cp.value.lower()}_{model_buf.pp_buf.prio_order.value.lower()}-{model_buf.pp_buf.pp.value.lower().replace(' ', '-')}_cr-{int(model_buf.coupled_removal)}'))

### Evaluation of `CacheMonitor` on `ContainerLogistics.json`

In [ ]:
for cp, pp, cr in [(CachePolicy.FIFO, PPBEventsPerObjectType(PrioPolicyOrder.MAX), False), (CachePolicy.LFU, PPBEventsPerObjectType(PrioPolicyOrder.MIN), True)]:
    pp.buf.clear()
    ocdfg_buf = OcdfgBuffer(
        node_buf_size=125, 
        arc_buf_size=125,
        cp=cp,
        pp_buf=pp,
        coupled_removal=cr,
        cache_monitor=CacheMonitor()
    )

    ocpn_buf = OcpnBuffer(
        node_buf_size=125, 
        arc_buf_size=125,
        ea_buf_size=125,
        cp=cp,
        pp_buf=pp,
        coupled_removal=cr,
        cache_monitor=CacheMonitor()
    )

    totem_buf = TotemBuffer(
        tr_buf_size=125, 
        ec_buf_size=125,
        lc_buf_size=125,
        cp=cp,
        pp_buf=pp,
        coupled_removal=cr,
        cache_monitor=CacheMonitor()
    )

    for model_buf, model in [(totem_buf, 'totem'), (ocdfg_buf, 'ocdfg'), (ocpn_buf, 'ocpn')]:
        model_buf.process_stream(event_stream.stream)
        model_buf.create_monitor_dataframes()
        create_all_plots_cache_monitor(model_buf.c_mon, Path(f'../figures/ContainerLogistics/{model}_monitoring/{cp.value.lower()}_{pp.prio_order.value.lower()}-{pp.pp.value.lower().replace(' ', '-')}_cr-{int(cr)}'))

        if model == 'totem':
            visualize_totem_overlap(
                TotemModel(totem_buf),
                TotemModel('../data/ContainerLogistics.xml'),
                output_dir=Path(f'../figures/ContainerLogistics/{model}_monitoring/{cp.value.lower()}_{pp.prio_order.value.lower()}-{pp.pp.value.lower().replace(' ', '-')}_cr-{int(cr)}'),
                output_file='totem_onl_vs_offl_overlap.pdf',
                ot_to_hex_color=ot_to_hex_color
            )